In [ ]:
import asyncio
import websockets
import nest_asyncio

import json

async def validate(uri, request):
    async with websockets.connect(uri) as websocket:
        await websocket.send(request)
        i = 1
        data = []
        while True:
            try:
                response = await websocket.recv()
                data.append(response)
            except websockets.ConnectionClosed:
                return data
            print('Receiving... ' + response + ' [' + str(i) + ']')
            i = i + 1

async def validateNetCdf( path, startsWith, endsWith, columns ):
    nest_asyncio.apply()
    
    loop = asyncio.get_running_loop()

    # Create a new Future object.
    fut = loop.create_future()

    request = { 'dir' : path, 'startsWith' : startsWith, 'endsWith' : endsWith, 'expectedColumns' : columns } 
    j = json.dumps(request)

    fut.set_result(await validate('ws://localhost:9000/validateasync', j))

    return await fut

path = 'C:\\Earthwave\\MountainGlaciers\\files'
startsWith = 'CS'
endsWith = '.nc'
columns = ['coh','power']

data = await validateNetCdf( path, startsWith, endsWith, columns )


In [ ]:
print(data[5])
    

    

